In [ ]:
# activate line execution
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# general
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import os
import shutil
import pickle

# plotly
import plotly.express as px  # (version 4.7.0 or higher)
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# import custom libraries
import sys
sys.path.append("C:\\DATA\\Tasks\\lib\\hk")
import hk_utils

# folder paths
ADD_DATA = "C:\\DATA\\data\\raw\\mimic4\\lookup\\"
ADD_DATA_proc = "C:/DATA/data/processed/"





In [ ]:
# libraries for THP

import argparse
import numpy as np
import pickle
import time
import torch
import torch.nn as nn
import torch.optim as optim
# from torch.utils.tensorboard import SummaryWriter

import transformer.Constants as Constants
import Utils

# from preprocess.Dataset import get_dataloader, get_dataloader2
# from transformer.Models import Transformer
# from transformer.hk_transformer import Transformer
from tqdm import tqdm

# from torchinfo import summary

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

# torch.cuda.empty_cache()
# torch.cuda.memory_allocated()
# torch.cuda.memory_reserved()

from sklearn import metrics
# from hk_pytorch import save_checkpoint,load_checkpoint
# import hk_pytorch


# from custom2 import myparser


In [ ]:
str_add = 'example.pkl'
str_add = 'example_so_new.pkl'
with open(str_add, 'rb') as f:
    example = pickle.load(f)

num_samples = 100
i_batch = 1

In [ ]:
# example.keys()
for key,item in example.items():
    print(key,"\t\t\t",item.shape)

# diff_time = (example['event_time'][:, 1:] - example['event_time'][:, :-1]) *  example['non_pad_mask'][:, 1:] # [B,L-1]
# unbiased_integral =  torch.sum(example['all_lambda'], dim=2) / num_samples * diff_time.unsqueeze(-1) # [B,L-1,K] 



In [ ]:
example['type_mask'].shape
example['type_mask'].transpose(0,2,1)[:,None,:,:].shape
example['self_attn'].shape
E_t = example['type_mask'].transpose(0,2,1)[:,None,:,:]
E = example['type_mask'][:,None,:,:]
att_agg = E_t @ example['self_attn'] @ E
att_agg.shape

In [ ]:
K = example['all_lambda'].shape[-1]
example['all_lambda'].shape

def cal_intensity_time(all_lambda, int_time, example):
    i = all_lambda # [B,L-1,ns,K]
    i = i * example['non_pad_mask'][:,:-1,None,None]
    i = i[i_batch].reshape(-1,K)
    # i.shape

    # int_time.shape
    t = (example['event_time'][:,[0],None]+int_time) # [B,L,ns]
    # t = t * example['non_pad_mask'][:,:-1,None]
    t = t[i_batch].flatten() # [L*ns]
    # t.shape




    all_lambda_int = np.sum(all_lambda, axis=2) / num_samples # [B,L-1,K]
    all_lambda_int.shape



    # i_batch=2
    # example['event_time'][:,0]

    t = (example['event_time'][i_batch,0,None]+int_time[i_batch]) # [B,L-1,ns]

    t = t.flatten() # [L*ns]

    return i,t

# Vis intensity

In [ ]:
#

# t[::10]
# i[::10,0]

# go.Figure(data=go.Scatter(x=t[::10], y=i[::10,0], mode='markers')).show()
# go.Figure(data=go.Scatter(x=t[::10], y=i[::10,2], mode='markers')).show()
K_limit = K
fig = make_subplots(rows=int(K_limit/3)+1, cols=3, shared_xaxes=True)
step=1

i, t = cal_intensity_time(example['all_lambda'], example['int_time'], example)
i_B, t = cal_intensity_time(example['all_lambda_B'], example['int_time'], example)
i_H, t = cal_intensity_time(example['all_lambda_H'], example['int_time'], example)
i_BH, t = cal_intensity_time(example['all_lambda_BH'], example['int_time'], example)

for ii in range(K_limit):
    
    row = int(ii/3)+1
    col = ii%3 + 1

    # plot intensity
    x = t[::step]

    ind = x.argsort()
    x=x[ind]
    # x.min()
    
    y=i[::step,ii][ind]
    _ = fig.add_trace(
        go.Scatter(x=x, y=y, mode='lines',marker_color='black', name='int'),
        row=row, col=col,
    )
    
    y=i_B[::step,ii][ind]
    _ = fig.add_trace(
        go.Scatter(x=x, y=y, mode='lines',marker_color='yellow', name='int_B'),
        row=row, col=col,
    )
    
    y=i_H[::step,ii][ind]
    _ = fig.add_trace(
        go.Scatter(x=x, y=y, mode='lines',marker_color='orange', name='int_H'),
        row=row, col=col,
    )
    
    y=i_BH[::step,ii][ind]
    _ = fig.add_trace(
        go.Scatter(x=x, y=y, mode='lines',marker_color='purple', name='int_BH'),
        row=row, col=col,
    )
    
    # example['event_time'][i_batch,:3]
    # x
    # y
    
    # print(np.trapz(y,x=x), all_lambda_int[i_batch,0,ii].sum(), example['unbiased_integral'][i_batch,0,ii].sum() )
    # term
    print(np.trapz(y,x=x)/ example['unbiased_integral'][i_batch,:,ii].sum())

    # plot occured events
    
    
    
    
    
    
    x = example['event_time'][i_batch][example['event_type'][i_batch]==ii+1]

    y = example['type_lambda'][i_batch][example['event_type'][i_batch]==ii+1].flatten()
    _ = fig.add_trace(
        go.Scatter(x=x, y=y, mode='markers',marker_color='black', name='type_lam'),
        row=row, col=col
    )
    
    y = example['type_lambda_B'][i_batch][example['event_type'][i_batch]==ii+1].flatten() 
    _ = fig.add_trace(
        go.Scatter(x=x, y=y, mode='markers',marker_color='yellow', name='type_lam_B'),
        row=row, col=col
    )
    
    y = example['type_lambda_H'][i_batch][example['event_type'][i_batch]==ii+1].flatten() 
    _ = fig.add_trace(
        go.Scatter(x=x, y=y, mode='markers',marker_color='orange', name='type_lam_H'),
        row=row, col=col
    )

    y = example['type_lambda_BH'][i_batch][example['event_type'][i_batch]==ii+1].flatten() 
    _ = fig.add_trace(
        go.Scatter(x=x, y=y, mode='markers',marker_color='purple', name='type_lam_BH'),
        row=row, col=col
    )

# for y in example['event_time'][i_batch]:
#     type(y)
#     _ = fig.add_vline(x=y,row='all', col=1)

_ = fig.update_xaxes(matches='x')
_ = fig.update_layout(hovermode="x unified")
fig.show()
fig.write_html(ADD_DATA+"/intensity.html")


# Vis att

In [ ]:
max_len = 5
max_len = example['self_attn'].shape[-1]
nheads = example['self_attn'].shape[1]


fig = make_subplots(rows=1, cols=nheads,shared_xaxes=True)

for i_head in range(nheads):
    
    mat = example['self_attn'][i_batch,i_head] * (~example['slf_attn_mask'][i_batch])  
    
    mat = mat * (np.arange(mat.shape[0])+1)[:,None]
    
    mat = mat[:max_len,:max_len]

    # mat[mat<1.05]=0
    labels = [f"E{i}_{event}" for i,event in enumerate(example['event_type'][i_batch])] [:max_len]
    # mat = mat / np.sum(mat,axis=1,keepdims=True)
    
    
    # fig2 = px.imshow(mat,x = labels,y = labels).update_layout(width=500, height=500)
    # fig2.show()
    x = labels.copy()
    y = labels.copy()
    
    y.reverse()


    mat2= np.flip(mat,axis=0)
    _ = fig.add_trace(
        go.Heatmap(z=mat2, x= x, y=y),
        row = 1, col=i_head+1,
        
    )

fig.show()

# #     # _ = fig.add_trace(go.Heatmap(z=mat),
# #     #     row=i_head+1, col=1).update_layout(width=500, height=500)
# # # fig.update_layout(width=500, height=500)
# # # fig.show()

fig.write_html(ADD_DATA+"/att.html")


# Vis aggregation

In [ ]:
a = example['event_time'].flatten()
event_gap = (example['event_time'][:,1:] - example['event_time'][:,:-1] ) * example['non_pad_mask'][:,1:]
# event_gap[0]
# example['event_time'][0]
event_gap= event_gap[event_gap>0]

event_gap.mean()

In [ ]:
x=np.arange(10)
y=np.arange(9,-1,step=-1)
x
y

In [ ]:
import sklearn
fig = make_subplots(rows=int(K_limit/3)+1, cols=3, shared_xaxes=True)

example['type_mask'].shape
example['type_mask'].transpose(0,2,1)[:,None,:,:].shape
example['self_attn'].shape
E_t = example['type_mask'].transpose(0,2,1)[:,None,:,:]
E = example['type_mask'][:,None,:,:]
att_agg = E_t @ example['self_attn'] @ E
att_agg.shape
norm = (E_t @ E)
norm.shape
norm = norm.sum(0).sum(0)
np.diag(norm)
(np.diag(norm) / np.diag(norm).sum() *100).astype(int)
# np.diag(norm[0,0])
# norm[0,0]
# np.linalg.inv(norm[0,0])

list_temp = [[1., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 1., 0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 1., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 1., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1., 1., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0., 0., 1., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 1.]]
gt = np.array(list_temp)

gt= np.ones((K,K))


gt[gt>0]=1
gt

x=np.arange(K)
y=np.arange(K-1,-1,step=-1)
x=[str(i) for i in x]
y=[str(i) for i in y]
fig = make_subplots(rows=3, cols=4, shared_xaxes=True)

for i_head in range(nheads):
    


    print('att_agg')
    mat = att_agg[i_batch,i_head,:,:]
    mat = att_agg[:,i_head,:,:].sum(0)  # sum across batches
    _ = fig.add_trace(
        go.Heatmap(z=np.flip(mat, axis=0), x=x, y=y),
        row = 1, col=i_head+1

    )
    fpr, tpr, thresholds = metrics.roc_curve(gt.flatten(), mat.flatten(), pos_label=1)
    metrics.auc(fpr, tpr) 

    # print('att_agg new')
    # mat =  np.linalg.inv(norm)[None,:,:] @ att_agg[:,i_head,:,:] @ np.linalg.inv(norm)[None,:,:]*1000
    # mat = mat.sum(0)
    # _ = fig.add_trace(
    #     go.Heatmap(z=np.flip(mat, axis=0), x=x, y=y),
    #     row = 2, col=i_head+1,
        
    # )    
    # fpr, tpr, thresholds = metrics.roc_curve(gt.flatten(), mat.flatten(), pos_label=1)
    # metrics.auc(fpr, tpr)  



print('A_reg')
mat =  example['A_reg']

_ = fig.add_trace(
    go.Heatmap(z=np.flip(mat, axis=0), x=x, y=y),
    row = 3, col=1
)
fpr, tpr, thresholds = metrics.roc_curve(gt.flatten(), mat.flatten(), pos_label=1)
metrics.auc(fpr, tpr)

print('gt')
mat =  gt
_ = fig.add_trace(
    go.Heatmap(z=np.flip(mat, axis=0), x=x, y=y),
    row = 3, col=2
)


_ = fig.update_xaxes(matches='x')
fig.show()


fig.write_html(ADD_DATA+"/inf.html")


# Vis masks

In [ ]:

# self attention mask
mat = (~example['slf_attn_mask'][i_batch]).astype(int) [:max_len,:max_len]
fig = px.imshow(mat,x = labels,y = labels).update_layout(width=500, height=500)
fig.show()

# regulariztion mask
mat = (example['mask2'][i_batch]) [:max_len,:max_len]
fig = px.imshow(mat,x = labels,y = labels).update_layout(width=500, height=500)
fig.show()

# A_reg
mat = example['A_reg']
fig = px.imshow(mat).update_layout(width=500, height=500)
fig.show()